# Comparative Assignment
* __Theodoro Gasperin Terra Camargo__
* __r0974221__

## PIPELINE:
- Find protein of interest Homologs (BLASTP or HMMER3)
- Retrive species names from homologs
- Retrive as COI-5P seqs from BOLD database
- MSA (multiple sequence alignment)
- Phylogenetic Tree 

In [3]:
%pwd

'/Users/theo/Desktop/3rd_semester/Comparative_&_Regulatory_Genomics/Comparative/assignment/scripts'

In [4]:
%cd ../data

/Users/theo/Desktop/3rd_semester/Comparative_&_Regulatory_Genomics/Comparative/assignment/data


## Retrive homologs sequences from HMMER3.

HMMER3 is a bioinformatics tool that uses Hidden Markov Models (HMMs) to identify homologous sequences by detecting conserved motifs and patterns in sequence alignments. Unlike BLASTP, which is alignment-based, HMMER3 focuses on statistical models for greater sensitivity in detecting remote homologs, particularly in protein families with subtle similarities. Its advantage over BLASTP lies in its precision for identifying weakly conserved sequences, though it is typically slower due to computational intensity. HMMER3 is particularly effective in large-scale domain and protein family analyses.

Using HMMER3 (https://www.ebi.ac.uk/jdispatcher/sss/hmmer3_phmmer), I performed a sequence similarity search for the Human HOXD13 protein. 
The results from this search can be found here: https://www.ebi.ac.uk/jdispatcher/sss/hmmer3_phmmer/summary?jobId=hmmer3_phmmer-I20250111-155037-0551-75430741-p1m

### Retriving fasta sequences for homologs

In [5]:
import requests

def fetch_fasta_multiple(swissprot_ids, output_file="sequences.fasta"):
    """
    Fetches FASTA sequences for a list of SwissProt IDs and saves them to a single file.

    Parameters:
    swissprot_ids (list): A list of SwissProt IDs to fetch sequences for.
    output_file (str): The name of the file to save the sequences to.

    Returns:
    None
    """
    base_url = "https://www.uniprot.org/uniprot"
    
    with open(output_file, "w") as file:
        for swissprot_id in swissprot_ids:
            url = f"{base_url}/{swissprot_id}.fasta"
            response = requests.get(url)
            if response.status_code == 200:
                file.write(response.text)
            else:
                print(f"Error retrieving FASTA for {swissprot_id}: {response.status_code}")

    print(f"FASTA sequences saved to {output_file}")

In [120]:
ids = []
with open("HOXD13/HMMer/hmmer3_phmmer-I20250111-155037-0551-75430741-p1m.ids", "r") as file:
    for line in file:
        ids.append(line.split("|")[1])

# Fetch the sequences in Uniprot
fetch_fasta_multiple(ids, "HOXD13/HMMer/hmmer_homologs.fasta")

FASTA sequences saved to HOXD13/HMMer/hmmer_homologs.fasta


## Retrive species list from our Homologs 

In [64]:
def extract_species_names(fasta_file):
    '''
    Retrives the species names from a FASTA file.
    returns a list of species names.
    '''

    species_names = []

    with open(fasta_file, "r") as file:
        for line in file:
            if line.startswith(">"):  # Check if the line is a header   
                if "(Fragment)" in line:
                    line = line.split("(Fragment)")[0] + line.split("(Fragment)")[1]
                name = line.split()[4] + " " + line.split()[5]
                species_names.append(name[3:])
    
    return species_names

In [66]:
# Extracting species names from the FASTA file 
fasta_file = "HOXD13/HMMer/hmmer_homologs.fasta"
species_list = extract_species_names(fasta_file)

unique_species = set(species_list)
print(f"Number of unique species: {len(unique_species)}")
print(unique_species)

Number of unique species: 25
{'Macaca nemestrina', 'Heterodontus francisci', 'Physeter macrocephalus', 'Notophthalmus viridescens', 'Macaca mulatta', 'Pan troglodytes', 'Takifugu rubripes', 'Ambystoma mexicanum', 'Carollia perspicillata', 'Pongo pygmaeus', 'Morone saxatilis', 'Gallus gallus', 'Ovis aries', 'Devario aequipinnatus', 'Homo sapiens', 'Mus musculus', 'Danio rerio', 'Saguinus labiatus', 'Cavia porcellus', 'Lagothrix lagotricha', 'Oryzias latipes', 'Tripneustes gratilla', 'Caenorhabditis elegans', 'Marmota monax', 'Xenopus laevis'}


We are required to have a minimum of 25 species for our analysis
Therefore, we will add two more species to our homologs file.

In [19]:
new_species_list = ['Marmota monax', 'Physeter macrocephalus']
for sp in new_species_list:
    if sp in species_list:
        print(f"{sp} is already in the list")
    else:
        print(f"{sp} is not in the list")
        # species_list.append(sp)

Eschrichtius robustus is not in the list
Marmota monax is already in the list
Physeter macrocephalus is already in the list
Danio rerio is already in the list


I then proceed to add the the sequences for HOXD13 for both this species to our hhmer_homologs.fasta file

\>tr|A0A5E4BI81|A0A5E4BI81_MARMO Homeobox protein Hox-D13 OS=Marmota monax OX=9995 GN=GHT09_012440 PE=3 SV=1
MDGLRADAGAAGAAPASSSSSVAAAAAPGQCRGFLSAPVFAGTHSGRAAAAAAAAAAAAA
AASSFAYPGTSERTGSASSSSSSAVVAARPEAPPAKECTAPAPTAAAAAPPGAPALGYGY
HFGNGYYSCRMSHGVGLQQNALKSSPHASLGGFPVEKYMDVSGLASSSVPTNEVPARAKE
VSFYQGYTSPYQHVPGYIDMVSTFGSGEPRHEAYISMEGYQSWTLANGWNSQVYCAKDQP
QGSHFWKSSFPGDVALNQPDMCVYRRGRKKRVPYTKLQLKELENEYAINKFINKDKRRRI
SAATNLSERQVTIWFQNRRVKDKKIVSKLKDTVS
\>tr|A0A2Y9F0G6|A0A2Y9F0G6_PHYMC Homeobox protein Hox-D13 OS=Physeter macrocephalus OX=9755 GN=HOXD13 PE=3 SV=1
MSRAGSWDMDGLRADGGAAGAAPASSSSSSVAAAAPGQCRGFLSAPVFAGTHSGRAAAAA
AAAAAAAAAAAASGFAYPGTSERAGSASSSSSSAVVAARPEASSAKECPAPGAAAAAPPG
APALGYGYHFGNGYYSCRMSHGVGLQQNALKSSPHASLGGFPVEKYMDVSGLASSSVPAN
EVPARAKEVSFYQGYTSPYQHVPGYIDMVSTFGSGEPRHEAYISMEGYQSWTLANGWNSQ
VYCAKDQPQGSHFWKSSFPGDVALNQPDMCVYRRGRKKRVPYTKLQLKELENEYAINKFI
NKDKRRRISAATNLSERQVTIWFQNRRVKDKKIVSKLKDTVS

## Retrive COI-5P genes from our list of species

COI-5P refers to a specific region of the cytochrome c oxidase I (COI or COX1) gene, which is part of the mitochondrial genome. This region is widely used as a DNA barcode for species identification and biodiversity studies, particularly in animals. In this study, we utilize the BOLD Systems API to retrieve the COI-5P gene from a list of species. The BOLD Systems API (Barcode of Life Data Systems) provides programmatic access to the data and functionality of the BOLD Systems platform, a central repository for DNA barcode data and related biological information. This API enables users to interact with BOLD data programmatically for tasks such as searching for species, retrieving barcode sequences, and accessing specimen data.

In [44]:
import requests
from Bio import SeqIO

# Function to fetch COI-5P sequences for a species
def query_id(species_name):
    # Define the API base URL
    API_BASE_URL = "https://portal.boldsystems.org/api/query"

    params = {
        "query": "tax:species:" + species_name,
        "extent": "limited",  
    }

    response = requests.get(API_BASE_URL, params=params)
    
    if response.status_code == 200:
        try:
            return response.json()  # Parse the JSON response
        except ValueError:
            print(f"Failed to parse JSON for {species_name}")
            return None
    else:
        print(f"Error fetching data for {species_name}: {response.status_code}")
        return None
     
# Function to fetch COI-5P sequences for a species
def retrive_barcode(query_id):
    API_BASE_URL = "https://portal.boldsystems.org/api/documents/" + query_id

    params = {
        "length": 3,   
        "start": 0,
    }
    response = requests.get(API_BASE_URL, params=params)
    #print(response.url)
    
    if response.status_code == 200:
        try:
            return response.json()  # Parse the JSON response
        except ValueError:
            print(f"Failed to parse JSON for {query_id}")
            return None
    else:
        print(f"Error fetching data for {query_id}: Response status code = {response.status_code}")
        print()
        return None

# Save sequences to a FASTA file
def save_to_fasta(results, output_file):
    with open(output_file, "w") as fasta_file:
        for species in results.keys():
            fasta_file.write(f">{species}\n")
            fasta_file.write(f"{results[species]}\n")

def make_fasta_preaty(input_file, output_file):
    sequences = list(SeqIO.parse(input_file, "fasta"))
    SeqIO.write(sequences, output_file, "fasta")
    print(f"Modified FASTA file saved to {output_file}")

In [ ]:
output_fasta = "COI-5P/coi_5p_sequences_hmmer3.fasta"  # Output FASTA file
not_found = []

results = {}
for species in unique_species:
    print(f"Searching for {species}...")
    query = query_id(species)
    
    if query:
        data = retrive_barcode(query['query_id'])
        if data:
            if data["recordsTotal"] == 0:
                print(f"No records found for {species}")
                not_found.append(species)
                continue
            else: # Multiple records found (I limited to 3 records only)
                for index, record in enumerate(data['data']):
                    if record['marker_code'] == "COI-5P":
                        print(f"Marker {record['marker_code']}")
                        results[species] = record['nuc']
                        break
        else: 
            print("Error: retrive_barcode failed!")
    else:
        print("Error: query_id failed!")

#  print(results)
save_to_fasta(results, output_fasta)
print(f"FASTA file saved to {output_fasta}")

Searching for Macaca nemestrina...
Marker COI-5P
Searching for Heterodontus francisci...
Marker COI-5P
Searching for Physeter macrocephalus...
Marker COI-5P
Searching for Notophthalmus viridescens...
Marker COI-5P
Searching for Macaca mulatta...
Marker COI-5P
Searching for Pan troglodytes...
Marker COI-5P
Searching for Takifugu rubripes...
Marker COI-5P
Searching for Ambystoma mexicanum...
Marker COI-5P
Searching for Carollia perspicillata...
Marker COI-5P
Searching for Pongo pygmaeus...
Marker COI-5P
Searching for Morone saxatilis...
Marker COI-5P
Searching for Gallus gallus...
Marker COI-5P
Searching for Ovis aries...
Marker COI-5P
Searching for Devario aequipinnatus...
Marker COI-5P
Searching for Homo sapiens...
Marker COI-5P
Searching for Mus musculus...
Marker COI-5P
Searching for Danio rerio...
Marker COI-5P
Searching for Saguinus labiatus...
Marker COI-5P
Searching for Cavia porcellus...
Marker COI-5P
Searching for Lagothrix lagotricha...
Marker COI-5P
Searching for Oryzias lati

Now we have our fasta file with all the COI-5P sequences from most of our species. 

In [70]:
%cat "COI-5P/coi_5p_sequences_hmmer3.fasta" | head -n 3

>Macaca nemestrina
ACAACGTTATTGTAACGGCCCACGCATTCATTATAATCTTCTTTACAGTCATACCTATTATGATTGGGGGTTTCGGGAATTGATTAGTGCCCTTAATAATCGGCGCACCCGACATGGCATTTCCCCGCCTAAATAACATAAGCTTCTGACTCCTCCCTCCTTCTTTCCTGCTACTAATAGCATCAGCCATAGTAGAAGCTGGCGCTGGAACAGGTTGAACAGTATACCCCCCTTTAGCAGGAAACTTCTCCCACCCAGGAGCTTCCGTAGACCTAATCATCTTCTCCCTCCACCTGGCAGGTATTTCCTCTATCCTAGGGGCCATCAACTTTATTACCACTATCATCAACATAAAACCCCCCGCAATATCCCTATACCAAACCCCTTTATTTGTCTGATCGATCTTAATCACAGCAGTCCTCTTACTCCTCTCCCTACCAGTCTTAGCCGCTGGCATTACCATACTGCTAACAGACCGCAACCTCAATACTACCTTTTTTGACCCCGTTGGAGGAGGAGACCCTATCCTATACCAACACC
>Heterodontus francisci


In [71]:
make_fasta_preaty("COI-5P/coi_5p_sequences_hmmer3.fasta", "COI-5P/coi_5p_preaty_hmmer3.fasta")

Modified FASTA file saved to COI-5P/coi_5p_preaty_hmmer3.fasta


In [72]:
%cat "COI-5P/coi_5p_preaty_hmmer3.fasta" | head 

>Macaca nemestrina
ACAACGTTATTGTAACGGCCCACGCATTCATTATAATCTTCTTTACAGTCATACCTATTA
TGATTGGGGGTTTCGGGAATTGATTAGTGCCCTTAATAATCGGCGCACCCGACATGGCAT
TTCCCCGCCTAAATAACATAAGCTTCTGACTCCTCCCTCCTTCTTTCCTGCTACTAATAG
CATCAGCCATAGTAGAAGCTGGCGCTGGAACAGGTTGAACAGTATACCCCCCTTTAGCAG
GAAACTTCTCCCACCCAGGAGCTTCCGTAGACCTAATCATCTTCTCCCTCCACCTGGCAG
GTATTTCCTCTATCCTAGGGGCCATCAACTTTATTACCACTATCATCAACATAAAACCCC
CCGCAATATCCCTATACCAAACCCCTTTATTTGTCTGATCGATCTTAATCACAGCAGTCC
TCTTACTCCTCTCCCTACCAGTCTTAGCCGCTGGCATTACCATACTGCTAACAGACCGCA
ACCTCAATACTACCTTTTTTGACCCCGTTGGAGGAGGAGACCCTATCCTATACCAACACC
cat: stdout: Broken pipe


## Removing alignment from COI-5P fasta sequences

In [76]:
def remove_gaps_from_fasta(input_file, output_file):
    """
    Removes '-' characters from every line in a FASTA file.

    Args:
        input_file (str): Path to the input FASTA file.
        output_file (str): Path to save the modified FASTA file.
    """
    with open(input_file, "r") as infile, open(output_file, "w") as outfile:
        for line in infile:
            if line.startswith(">"):
                outfile.write(line)
            else:
                outfile.write(line.replace("-", "").replace("N", ""))

    print(f"Gaps removed and saved to {output_file}")



In [78]:
# Example usage
input_fasta = "COI-5P/coi_5p_preaty_hmmer3.fasta"
output_fasta = "COI-5P/coi_5p_no_gaps_hmmer3.fasta"
remove_gaps_from_fasta(input_fasta, output_fasta)
make_fasta_preaty("COI-5P/coi_5p_no_gaps_hmmer3.fasta", "COI-5P/coi_5p_hmmer3.fasta")

Gaps removed and saved to COI-5P/coi_5p_no_gaps_hmmer3.fasta
Modified FASTA file saved to COI-5P/coi_5p_hmmer3.fasta


## Filtering species we could not found COI-5P gene!
We also have a list of missing species that we will need to filter from our HOXD13_homologs.fasta

In [89]:
print(f"Total species not found: {len(not_found)}")
print(f"species not found: {not_found}")

Total species not found: 15
species not found: ['Gorilla gorilla gorilla', 'Cebus imitator', 'Saimiri boliviensis boliviensis', 'Piliocolobus tephrosceles', 'Eulemur rufifrons', 'Cynocephalus volans', 'Enhydra lutris kenyoni', 'Gulo gulo luscus', 'Mustela putorius furo', 'Odobenus rosmarus divergens', 'Cnephaeus nilssonii', 'Ceratotherium simum simum', 'Hippopotamus amphibius kiboko', 'Sturnira hondurensis', 'Equus quagga']


In [98]:
def filter_not_found(homologs_file, species_list, output_file):
    with open(homologs_file, "rt") as file, open(output_file, "w") as out:
        write_sequence = False
        for line in file:
            if line.startswith(">"):  # Header line
                # Extract species name inside square brackets
                start = line.find("[") + 1
                end = line.find("]")
                name = ''
                if start > 0 and end > start:
                    name = line[start:end]
                if name not in species_list:
                    out.write(line)
                    write_sequence = True
                else:
                    write_sequence = False   
            elif write_sequence:
                out.write(line)  # Write sequence

In [99]:
# Filter not found sequences from our homologs file
homologs_file = "HOXD13/blastp_homologs.fasta"
output_file = "HOXD13/filtered_homologs.fasta"
filter_not_found(homologs_file, not_found, output_file)

### Renaming title lines in Homologs fasta file

In [34]:
from Bio import SeqIO

def change_fasta_titles(input_file, output_file):
    """
    Change the titles of sequences in a FASTA file.
    
    Args:
        input_file (str): Path to the input FASTA file.
        output_file (str): Path to save the modified FASTA file.
    """
    # Read sequences from the input FASTA file
    sequences = list(SeqIO.parse(input_file, "fasta"))

    # Modify the titles
    for i, record in enumerate(sequences):
        if "(Fragment)" in record.description:
            line = record.description
            line = line.split("(Fragment)")[0] + line.split("(Fragment)")[1]
            name = line.split()[4].strip("OS=") + "_" + line.split()[5]
            gene = line.split()[7].strip("GN=")
            record.id = name + "_" + gene
            record.description = ""
        else:
            line = record.description
            name = line.split()[4].strip("OS=") + "_" + line.split()[5]
            gene = line.split()[7].strip("GN=")
            record.id = name + "_" + gene
            record.description = ""
    
    # Save the modified sequences to the output file
    SeqIO.write(sequences, output_file, "fasta")
    print(f"Modified FASTA file saved to {output_file}")

In [35]:
# Example usage
input_fasta = "HOXD13/HMMer/hmmer_homologs.fasta"
output_fasta = "HOXD13/HMMer/HOXD13_hmmer_homologs.fasta"
change_fasta_titles(input_fasta, output_fasta)

Modified FASTA file saved to HOXD13/HMMer/HOXD13_hmmer_homologs.fasta


## MSA (Multiple Sequence Alignment)
we will use MUSCLE for our sequences alignments

__Aligning our COI-5P sequences__

In [80]:
import subprocess

# Define the MUSCLE command to run
muscle_command = ["muscle", "-align", "COI-5P/coi_5p_hmmer3.fasta", "-output", "COI-5P/alignment_coi_5p_hmmer3.fasta"]

# Run the command
subprocess.run(muscle_command)


muscle 5.3.osxarm64 [-]  8.6Gb RAM, 8 cores
Built Dec 12 2024 04:49:44
(C) Copyright 2004-2021 Robert C. Edgar.
https://drive5.com

[align COI-5P/coi_5p_hmmer3.fasta]
Input: 25 seqs, avg length 709, max 1557, min 255

00:00 4.0Mb   100.0% Derep 25 uniques, 0 dupes
00:00 4.0Mb  CPU has 8 cores, running 8 threads
00:03 281Mb   100.0% Calc posteriors
00:03 281Mb   100.0% UPGMA5         
00:03 327Mb   100.0% Consistency (1/2)
00:03 335Mb   100.0% Consistency (2/2)
00:04 335Mb   100.0% Refining         


CompletedProcess(args=['muscle', '-align', 'COI-5P/coi_5p_hmmer3.fasta', '-output', 'COI-5P/alignment_coi_5p_hmmer3.fasta'], returncode=0)

__Aligning our Homologs sequences__

In [36]:
import subprocess

# Define the MUSCLE command to run
muscle_command = ["muscle", "-align", "HOXD13/HMMer/HOXD13_hmmer_homologs.fasta", "-output", "HOXD13/HMMer/alignment_HOXD13_hmmer_homologs.fasta"]

# Run the command
subprocess.run(muscle_command)


muscle 5.3.osxarm64 [-]  8.6Gb RAM, 8 cores
Built Dec 12 2024 04:49:44
(C) Copyright 2004-2021 Robert C. Edgar.
https://drive5.com

[align HOXD13/HMMer/HOXD13_hmmer_homologs.fasta]
Input: 102 seqs, avg length 283, max 388, min 60

00:00 4.1Mb   100.0% Derep 101 uniques, 1 dupes
00:00 4.6Mb  CPU has 8 cores, running 8 threads
00:11 342Mb   100.0% Calc posteriors
00:11 343Mb   100.0% UPGMA5         
00:26 173Mb   100.0% Consistency (1/2)
00:41 185Mb   100.0% Consistency (2/2)
00:42 231Mb   100.0% Refining         


CompletedProcess(args=['muscle', '-align', 'HOXD13/HMMer/HOXD13_hmmer_homologs.fasta', '-output', 'HOXD13/HMMer/alignment_HOXD13_hmmer_homologs.fasta'], returncode=0)

## Phylogenetic Tree

To create the phylogenetic tree, I used MEGA11. I uploaded our MUSCLE MSAs to MEGA and converted them from .fasta to .meg format. 

Next, I chose to use Neighbor Joining to construct or tree

In [43]:
from Bio import SeqIO

def change_fasta_titles2(input_file, output_file):
    """
    Change the titles of sequences in a FASTA file.
    
    Args:
        input_file (str): Path to the input FASTA file.
        output_file (str): Path to save the modified FASTA file.
    """
    # Read sequences from the input FASTA file
    sequences = list(SeqIO.parse(input_file, "fasta"))

    # Modify the titles
    for i, record in enumerate(sequences):
        name = record.description.split(";")[-1]
        start = name.find("(") + 1
        end = name.find(")")
        if start > 0 and end > start:
            record.id = name[:start - 2].replace(" ", "_")
            record.description = name[start - 1:end + 1]
        else:
            record.id = name.replace(" ", "_")
            record.description = ""
    
    # Save the modified sequences to the output file
    SeqIO.write(sequences, output_file, "fasta")
    print(f"Modified FASTA file saved to {output_file}")

In [2]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

def merge_scaffolds(input_file, output_file, overlap_size=20):
    """
    Merge scaffolds based on overlapping sequences.
    
    Args:
        input_file (str): Path to the input FASTA file containing scaffolds.
        output_file (str): Path to save the merged sequence.
        overlap_size (int): Minimum overlap size between sequences.
    """
    # Read scaffolds from the input FASTA file
    scaffolds = list(SeqIO.parse(input_file, "fasta"))
    merged_sequence = str(scaffolds[0].seq)

    for i in range(1, len(scaffolds)):
        current_seq = str(scaffolds[i].seq)
        # Search for overlap between the end of the merged sequence and the start of the current sequence
        for j in range(overlap_size, len(current_seq)):
            if merged_sequence.endswith(current_seq[:j]):
                merged_sequence += current_seq[j:]
                break
        else:
            # If no overlap is found, append the sequence with a warning
            print(f"No overlap found for scaffold: {scaffolds[i].id}. Appending directly.")
            merged_sequence += current_seq

    # Save the merged sequence to the output file
    merged_record = SeqRecord(Seq(merged_sequence), id="merged_18S", description="Merged 18S rRNA sequence")
    SeqIO.write(merged_record, output_file, "fasta")
    print(f"Merged sequence saved to {output_file}")




In [3]:
# Example usage
input_fasta = "../data/18S_rRNA/Hyaena_hyaena/hyaena_scaffolds.fasta"
output_fasta = "../data/18S_rRNA/Hyaena_hyaena/hyaena_18S.fasta"
merge_scaffolds(input_fasta, output_fasta)

No overlap found for scaffold: NW_024082472.1:130934-131470. Appending directly.
No overlap found for scaffold: NW_024080745.1:2630726-2630972. Appending directly.
No overlap found for scaffold: NW_024080938.1:1146319-1146444. Appending directly.
No overlap found for scaffold: NW_024080938.1:1146296-1146324. Appending directly.
No overlap found for scaffold: NW_024082877.1:45720-45842. Appending directly.
No overlap found for scaffold: NW_024081557.1:238374-238406. Appending directly.
Merged sequence saved to ../data/18S_rRNA/Hyaena_hyaena/hyaena_18S.fasta


## MSA of Scaffolds 

We are gonna use MUSCLE for aligning the multiple scaffold into a single fasta file.

In [27]:
%cd ../data/18S_rRNA/Hyaena_hyaena/


/Users/theo/Desktop/3rd_semester/Comparative_&_Regulatory_Genomics/Comparative/assignment/data/18S_rRNA/Hyaena_hyaena


In [28]:
%pwd

'/Users/theo/Desktop/3rd_semester/Comparative_&_Regulatory_Genomics/Comparative/assignment/data/18S_rRNA/Hyaena_hyaena'

In [30]:
import subprocess

# Define the MUSCLE command to run
muscle_command = ["muscle", "-align", "scaffolds.fasta", "-output", "aligned_scaffolds.fasta"]

# Run the command
subprocess.run(muscle_command)


muscle 5.3.osxarm64 [-]  8.6Gb RAM, 8 cores
Built Dec 12 2024 04:49:44
(C) Copyright 2004-2021 Robert C. Edgar.
https://drive5.com

[align scaffolds.fasta]
Input: 5 seqs, avg length 2392649, max 5912046, min 39339


---Fatal error---
Too long, not appropriate for global alignment


CompletedProcess(args=['muscle', '-align', 'scaffolds.fasta', '-output', 'aligned_scaffolds.fasta'], returncode=1)

In [23]:
scaffolds = []

# Read the MSA file
with open("../data/18S_rRNA/Hyaena_hyaena/scaffolds_MSA.fasta") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        scaffolds.append(record)

for scaffold in scaffolds:
    print(len(scaffold.seq), scaffold.id)

# Calculate the consensus sequence
consensus = list(scaffolds[0].seq)
for i in range(1, len(scaffolds)):
    for j in range(len(consensus)):
        if scaffolds[i].seq[j] != "-":
            consensus[j] = scaffolds[i].seq[j]

# turn list back to string
consensus = "".join(consensus)
#print(consensus)

# # Save the consensus sequence to a FASTA file
consensus_record = SeqRecord(consensus, id="consensus_18S", description="Consensus 18S rRNA sequence")
output_file = "../data/18S_rRNA/Hyaena_hyaena/consensus_18S.fasta"
SeqIO.write(consensus_record, output_file, "fasta")
print(f"Consensus sequence saved to {output_file}")

619 NW_024080938.1:1146319-1146444
619 NW_024080938.1:1146296-1146324
619 NW_024083077.1:2506-2956
619 NW_024080745.1:2630726-2630972
619 NW_024081557.1:238374-238406
619 NW_024082472.1:130934-131470
619 NW_024082877.1:45720-45842
